# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [14]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这是一幅展示大自然美景的照片。图片中有一条木制的栈道穿越在一片郁郁葱葱的草地中，直通远方。草地上的草呈现出生机勃勃的绿色，旁边还有些灌木和散落的树木。天空是晴朗的，有几朵轻柔的白云。这幅图给人一种宁静平和的感觉，是大自然的一次美丽展示。这样的场景可能是在一个自然保护区或者是某个公园中。整体上，这张图片是一个很好的户外休闲地点示例，适合徒步和享受自然风光。', role='assistant', function_call=None, tool_calls=None))


In [2]:
response.choices[0].message.content

'这幅图呈现了一个宁静美丽的自然景观。图中的主要特点是一条木制的栈道，它穿越在郁郁葱葱的草地上，引向远处看不到的终点。草地上覆盖着茂密的绿色植被，显示出生机勃勃的春夏季节。背景中天空呈现出澄清的蓝色，点缀着几朵轻盈的白云，增添了一种宁静和平和的氛围。\n\n整个场景光线充足，阳光和蓝天与绿色植被的对比令人感觉清新愉快。这样的环境可能是理想的徒步旅行地点，提供了亲近自然和放松心情的完美机会。整体上，这幅图传达了大自然的宁静与和谐，是观赏和体验自然美景的一种'

### 封装成一个函数 query_image_description

In [15]:
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI()  # 初始化 OpenAI 客户端
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [7]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这幅图是一张幽默搞笑的图片，展示了两种不同体型的狗。左边是一只长得像健美运动员一样的狗，形象被夸张地带有人类健美运动员的特征，包括发达的肌肉和强壮的身体。右边则是一只更普通、体态略显圆润的狗，看起来悠闲而自在。

图中带有文字描述，左边的文案是“16岁的我，工作后的我”，指出左侧的狗代表青春时期强壮的态势；右边的狗旁的文案则是描述一种休闲、放松的生活方式。

整体而言，这张图片通过对比两种极端的形象，用幽默的方式表达了人们在年轻时和工作后可能会经历的身体和生活状态的变化。它让人们以一种轻松的方式反思生活和时间的影响


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [19]:
from openai import OpenAI
import base64
import requests
import json

client = OpenAI()  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.xiaoai.plus/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

In [9]:
content = query_base64_image_description("./images/gdp_1980_2020.jpg")
print(content)

ConnectTimeout: HTTPSConnectionPool(host='api.openai.com', port=443): Max retries exceeded with url: /v1/chat/completions (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000190CA2C69E0>, 'Connection to api.openai.com timed out. (connect timeout=None)'))

#### 使用 GPT-4V 识别手写体笔记

![](./images/handwriting_0.jpg)

In [20]:
content = query_base64_image_description("./images/handwriting_0.jpg")
print(content)

这张图显示的是笔记本中的一页，内容涉及深度学习和机器学习模型的微调方法，尤其是在自然语言处理领域中的应用。具体内容包括：

1. **Prompt Tuning** - 提到了对于小模型的 Prompt Tuning (可能是 referring to Efficiently Tuned Model)，涉及将输入 X 转换为嵌入向量（Embedding）然后通过特定的转换（如 Transformer）来生成输出 Y。

2. **Prefix Tuning** - 这部分讨论了在预训练模型的基础上添加预设的前缀来进行微调，涉及修改 Transformer 模型中的权重 W。

3. **LoRA 修改权重** - LoRA（Low-Rank Adaptation 的缩写），是一种针对预训练模型的参数更新方法，其目的是在不显著增加模型大小的情况下，通过改变权重矩阵来增加模型的能力。记号 ΔW = AB, 展示了通过低秩矩阵 A 和 B 来实现权重调整，其中 A 和 B 的维度较小，便于计算并减少额外的参数数量。

4. 文本中还提及了 **LoRA** 相关的存储需求，如 LoRA 需要 78GB，而一个叫做 LAMA 的实例（不太清楚具体指什么，可能是特定的 LoRA 配置或者是某种比较基准）仅需要 48GB。

总之，这页内容主要包含了几种 NLP 模型微调技术的概念和方法描述。


#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [10]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

这幅图是一张幽默搞笑的图片，展示了两种不同体型的狗。左边是一只长得像健美运动员一样的狗，形象被夸张地带有人类健美运动员的特征，包括发达的肌肉和强壮的身体。右边则是一只更普通、体态略显圆润的狗，看起来悠闲而自在。

图中带有文字描述，左边的文案是“16岁的我，工作后的我”，指出左侧的狗代表青春时期强壮的态势；右边的狗旁的文案则是描述一种休闲、放松的生活方式。

整体而言，这张图片通过对比两种极端的形象，用幽默的方式表达了人们在年轻时和工作后可能会经历的身体和生活状态的变化。它让人们以一种轻松的方式反思生活和时间的影响

![](./images/handwriting_1.jpg)

In [21]:
content = query_base64_image_description("./images/test.jpg")
display(Markdown(content))

这张图片显示了一份手写的习题练习纸，主要是关于乘法和除法的计算。红圈标记的部分显示了两个乘法题目的计算过程和结果，但都被打了红色的叉，表示计算错误。在左手边，`71除以5`的商被错误地记为11，而实际应该是14余1。在右手边，`7乘以5`的结果错误地给出为40，正确答案应该是35。

这种练习纸常用于学校教育中，帮助学生通过反复练习来掌握基本的数学计算技能。红色的叉号表明老师或家长已经检查过这些答案并指出了错误。

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。